## Finding NH DB2 deleted users by doing a back scrawl search

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


2021-08-26 00:53:10,488| INFO    | 0 keys loaded from agent


2021-08-26 00:53:10,488 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


2021-08-26 00:53:10,493| INFO    | 0 key(s) loaded


2021-08-26 00:53:10,493 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


...

2021-08-26 00:53:10,495 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'saxenap'


2021-08-26 00:53:10,498 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:101]: Started SSH Tunnel with db2.nanohub.org


2021-08-26 00:53:10,807| INFO    | Opening tunnel: 0.0.0.0:54988 <> 127.0.0.1:3306


2021-08-26 00:53:10,807 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:54988 <> 127.0.0.1:3306


Obtained Salesforce access token ...... True


In [2]:
import sys
# sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import time
import datetime

# from DB2SalesforceAPI import DB2SalesforceAPI

## Pull data from SF

In [3]:
db_1 = salesforce

In [4]:
NH_ids_sf = db_1.query_data('Select ID, nanoHUB_user_ID__c, Name, Email from Contact')

[Success] Bulk job creation successful. Job ID = 7505w00000ZJLCBAA5


{"id":"7505w00000ZJLCBAA5","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-08-26T04:53:12.000+0000","systemModstamp":"2021-08-26T04:53:12.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000ZJLCBAA5","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-08-26T04:53:12.000+0000","systemModstamp":"2021-08-26T04:53:17.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1303,"retries":0,"totalProcessingTime":270}


{"id":"7505w00000ZJLCBAA5","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-08-26T04:53:12.000+0000","systemModstamp":"2021-08-26T04:53:17.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1303,"retries":0,"totalProcessingTime":270}


{"id":"7505w00000ZJLCBAA5","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-08-26T04:53:12.000+0000","systemModstamp":"2021-08-26T04:53:38.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":253574,"retries":0,"totalProcessingTime":12477}
[Success] Bulk job completed successfully.


In [5]:
display(NH_ids_sf.head(5))
print(NH_ids_sf.shape)

,Email,Id,Name,nanoHUB_user_ID__c
0,nkissebe@gmail.com,0035w000031Vsp1AAC,hubrepo hubrepo,998.0
1,support@nanohub.org,0035w000031Vsp2AAC,nanoHUB support,1683.0
2,gridstat@nanohub.org,0035w000031Vsp3AAC,Grid Statistics,1684.0
3,ncn@nanohub.org,0035w000031Vsp4AAC,NCN NCN,1685.0
4,apps@nanohub.org,0035w000031Vsp5AAC,nanoHUB nanoHUB,1686.0


(252271, 4)


## Pull data from DB2

In [6]:
sql_query = "select id, name, username, email from jos_users" ## this id is the NH user id

# display
print(sql_query)

select id, name, username, email from jos_users


In [7]:
df = pd.read_sql_query(sql_query, nanohub_db)

In [8]:
display(df.head(5))
print(df.shape)

,id,name,username,email
0,998,hubrepo hubrepo,hubrepo,nkissebe@gmail.com
1,1000,CMS Admin Manager,admin,support@nanohub.org
2,1683,nanoHUB support,support,support@nanohub.org
3,1684,Grid Statistics,gridstat,gridstat@nanohub.org
4,1685,NCN NCN,ncn,ncn@nanohub.org


(256371, 4)


## Find deleted DB2 folks via NH id 

In [9]:
NH_ids_sf = NH_ids_sf.dropna()
print(NH_ids_sf.shape)

(238465, 4)


In [10]:
# crossref nh ids and sf nh id
# reformat the SF nh ids as integers rather than floats
nh_uids = NH_ids_sf['nanoHUB_user_ID__c'].apply(lambda x: int(x))
nh_uids = nh_uids.tolist()

In [11]:
db2_uids = df['id'].tolist()

In [12]:
## find all SF NH_ids that are not in db2_uids
temp = set(nh_uids).intersection(db2_uids) #set(db2_uids).intersection(nh_uids)
temp_diff = set(nh_uids).difference(db2_uids) #set(db2_uids).difference(nh_uids)

In [13]:
print(len(temp))
print(len(temp_diff))

238252
213


In [14]:
temp_diff = sorted(list(temp_diff))

In [15]:
## map nh_id_val to corresponding NH_ids_sf dataframe's x row index

In [16]:
print(temp_diff[:20])

[8628, 10086, 10287, 10424, 10533, 10832, 12070, 12314, 20780, 20938, 23192, 24064, 28538, 28626, 31179, 32694, 35202, 35782, 41960, 44341]


In [17]:
x_inds = []

for i in temp_diff:
    try:
        x_inds.append(nh_uids.index(i))
    except:
        # duplicate emails so that index not there
        tt_temp = 1

In [18]:
print(len(x_inds))

213


In [19]:
nh_dels_df = NH_ids_sf.iloc[x_inds,:].reset_index()
display(nh_dels_df.head(5))

,index,Email,Id,Name,nanoHUB_user_ID__c
0,101575,dwsievers@gmail.com,0035w000034JJcfAAG,D Sievers,8628.0
1,107389,anoshraj@gmail.com,0035w000034JL8XAAW,anosh raj,10086.0
2,107981,dlr@ee.wustl.edu,0035w000034JLI6AAO,dan rode,10287.0
3,108686,willsd@appstate.edu,0035w000034JLTTAA4,steve williams,10424.0
4,108795,hmadanian@gmail.com,0035w000034JLVEAA4,Jam gord,10533.0


In [20]:
nh_dels_df = nh_dels_df.drop(columns=['index'])
display(nh_dels_df.head(5))

,Email,Id,Name,nanoHUB_user_ID__c
0,dwsievers@gmail.com,0035w000034JJcfAAG,D Sievers,8628.0
1,anoshraj@gmail.com,0035w000034JL8XAAW,anosh raj,10086.0
2,dlr@ee.wustl.edu,0035w000034JLI6AAO,dan rode,10287.0
3,willsd@appstate.edu,0035w000034JLTTAA4,steve williams,10424.0
4,hmadanian@gmail.com,0035w000034JLVEAA4,Jam gord,10533.0


In [21]:
dels_emails = ['deleted_'+i for i in nh_dels_df['Email'].tolist()]
# dels_names = ['deleted_'+i for i in nh_dels_df['Name'].tolist()]
dels_nhids = nh_dels_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))

In [22]:
nh_dels_df['Email'] = dels_emails
# nh_dels_df['Name'] = dels_names
nh_dels_df['nanoHUB_user_ID__c'] = dels_nhids

In [23]:
nh_dels_df = nh_dels_df.drop(columns=['Id','Name'])
nh_dels_df['DB2_deleted__c'] = True

In [24]:
display(nh_dels_df.head(5))

,Email,nanoHUB_user_ID__c,DB2_deleted__c
0,deleted_dwsievers@gmail.com,8628,True
1,deleted_anoshraj@gmail.com,10086,True
2,deleted_dlr@ee.wustl.edu,10287,True
3,deleted_willsd@appstate.edu,10424,True
4,deleted_hmadanian@gmail.com,10533,True


## push results to SF

In [25]:
db_2 = salesforce

In [26]:
db_2.object_id = 'Contact'
db_2.external_id = 'nanoHUB_user_ID__c'

In [27]:
db_2.send_data(nh_dels_df)

[Success] Bulk job creation successful. Job ID = 7505w00000ZJLCGAA5
hello


[Success] CSV upload successful. Job ID = 7505w00000ZJLCGAA5


[Success] Closing job successful. Job ID = 7505w00000ZJLCGAA5


In [28]:
# check status
db_2.check_bulk_status()

{'id': '7505w00000ZJLCGAA5',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-08-26T04:54:00.000+0000',
 'systemModstamp': '2021-08-26T04:54:01.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [29]:
# check status
from pprint import pprint

pprint(db_2.check_bulk_failed_results())

''
